## Getting cities,states and countries name from given list of coordinates


In [1]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def coordinates_to_location(coordinates):
    """
    Function to map list of geographical coordinates to city, state, and country.
    """
    geolocator = Nominatim(user_agent="geo_script")
    locations = []
    for coord in coordinates:
        try:
            location = geolocator.reverse(coord, exactly_one=True)
            locations.append(location)
        except GeocoderTimedOut:
            locations.append("Error: Geocoder Timed Out")
    return locations

# Example usage with coordinates for Dubai and another location
coordinates_list = [(25.0657000, 55.1712800), (24.45118, 54.39696)]  

# Mapping coordinates to city, state, and country
mapped_locations = coordinates_to_location(coordinates_list)

# Printing mapped locations
for location in mapped_locations:
    if isinstance(location, str):
        print(location)
    else:
        print("Country:", location.raw['address'].get('country', 'N/A'))
        print("City:", location.raw['address'].get('city', 'N/A'))
        print("State:", location.raw['address'].get('state', 'N/A'))


Country: الإمارات العربية المتحدة
City: N/A
State: دبي
Country: الإمارات العربية المتحدة
City: أبو ظبي
State: أبو ظبي


## Getting coordinates from cities,states and countries name


In [2]:
def location_to_coordinates(locations):
    """
    Function to map city names given in list to geographical coordinates.
    """
    geolocator = Nominatim(user_agent="geo_script")
    coordinates = []
    for loc in locations:
        try:
            location = geolocator.geocode(loc)
            if location:
                coordinates.append((location.latitude, location.longitude))
            else:
                coordinates.append("Error: Location not found")
        except GeocoderTimedOut:
            coordinates.append("Error: Geocoder Timed Out")
    return coordinates
# Example usage with city names for Al Ain and Ras Al Khaimah
city_names = ["Al Ain", "Ras Al Khaimah"]
# Mapping city names to coordinates
mapped_coordinates = location_to_coordinates(city_names)
# Printing mapped coordinates
for coord in mapped_coordinates:
    print("Coordinates:", coord)

Coordinates: (24.2248697, 55.7452211)
Coordinates: (25.7386349, 55.924922399798014)


In [ ]:
def coordinates_within_tolerance(coord1, coord2, tolerance):
    """
    Function to check if two coordinates fall within a specified tolerance range.
    """
    lat_diff = abs(coord1[0] - coord2[0])
    lon_diff = abs(coord1[1] - coord2[1])
    return lat_diff <= tolerance and lon_diff <= tolerance

def find_matching_coordinates(coords_list1, coords_list2, tolerance):
    """
    Function to find matching coordinates between two lists within a specified tolerance.
    """
    matching_pairs = []
    for coord1 in coords_list1:
        for coord2 in coords_list2:
            if coordinates_within_tolerance(coord1, coord2, tolerance):
                matching_pairs.append((coord1, coord2))
    return matching_pairs

# Example usage
coords_list1 = [(40.7128, -74.0060), (51.5074, -0.1278), (30.2672, -97.7431)]
coords_list2 = [(40.7128, -74.0060), (48.8566, 2.3522), (37.7749, -122.4194)]
tolerance = 0.1  # Tolerance range in degrees

matching_coordinates = find_matching_coordinates(coords_list1, coords_list2, tolerance)

# Printing matching coordinates
print("Matching coordinates within tolerance range:")
for pair in matching_coordinates:
    print("Coord1:", pair[0], "Coord2:", pair[1])


## Getting random coordinates of list of cities

In [1]:
import pandas as pd
from geopy.distance import geodesic
from random import uniform

# Define the list of cities with their coordinates
cities = {
    "New York": (40.7128, -74.0060),
    "Los Angeles": (34.0522, -118.2437),
    "Chicago": (41.8781, -87.6298),
    "Houston": (29.7604, -95.3698),
    "Phoenix": (33.4484, -112.0740),
    "Philadelphia": (39.9526, -75.1652),
    "San Antonio": (29.4241, -98.4936),
    "San Diego": (32.7157, -117.1611),
    "Dallas": (32.7767, -96.7970),
    "San Jose": (37.3382, -121.8863)
}

# Define the radius in kilometers
radius_km = 50

# Function to generate random coordinates within a radius around a city
def generate_coordinates(city):
    lat, lon = cities[city]
    distance = uniform(0, radius_km)
    bearing = uniform(0, 360)
    point = geodesic(kilometers=distance).destination((lat, lon), bearing)
    return point.latitude, point.longitude

# Generate 5000 random coordinates for each city
data = []
for city, (lat, lon) in cities.items():
    for _ in range(5000):
        lat_new, lon_new = generate_coordinates(city)
        data.append({"City": city, "Latitude": lat_new, "Longitude": lon_new})

# Create a DataFrame
df = pd.DataFrame(data)

# Export DataFrame to Excel
df.to_excel("coordinates.xlsx", index=False)


In [2]:
from pytube import YouTube
import re

# Download YouTube video
def download_video(url, save_path):
    yt = YouTube(url)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path=save_path)

# Extract location from video description or comments
def extract_location_from_text(text):
    # Example regex pattern to find location-related keywords
    pattern = r'\b(location|place|where|coordinates):?\s*([^\n\r]+)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(2)
    else:
        return None

# Example usage
video_url = 'https://youtu.be/MWrGuGDQXCE?si=IhA8mNYskDhVN8Oh'
save_path = r"D:\_2.Programming"
download_video(video_url, save_path)

# Extract location from video description or comments
# Assuming you have access to the video description or comments as text
video_description = "Your video description text here"
location = extract_location_from_text(video_description)
if location:
    print("Location:", location)
else:
    print("Location information not found.")


Location information not found.


In [ ]:
import requests
import json

def get_hotels(api_key, location, radius=5000, keyword='hotel'):
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {
        'key': api_key,
        'location': location,
        'radius': radius,
        'keyword': keyword
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['results']
    else:
        print("Error:", response.status_code)
        return None

def main():
    api_key = 'your_api_key'
    location = '25.276987,55.296249'  # Dubai coordinates
    radius = 5000  # 5 km radius around Dubai
    keyword = 'hotel'
    hotels = get_hotels(api_key, location, radius, keyword)
    if hotels:
        for hotel in hotels:
            print(hotel['name'], hotel['vicinity'])
    else:
        print("No hotels found.")

if __name__ == "__main__":
    main()
